In [1]:
import sys
sys.path.append('../../common/')
import pandas as pd
from functools import reduce

In [2]:
from crowddata_aggregator import CrowdsourceAggregator

In [3]:
batch = '5'

# Aggregate constructiveness annotations

In [4]:
# set paths for crowd annotated files
constructiveness_path = '../../CF_output/constructiveness/'
crowd_annotations_full = 'batch5_f1282373.csv'
input_csv = constructiveness_path + crowd_annotations_full

In [5]:
# Create a CrowdAggregator object
constructiveness_aggregator = CrowdsourceAggregator(input_csv)

In [6]:
# Dataframe with only non gold questions. 
constructiveness_df = constructiveness_aggregator.get_non_gold_questions()

In [7]:
# Set attributes
attribs = {}
# set attributes 
attribs['unit_id_col'] = ['_unit_id']
attribs['meta_cols'] = ['article_author', 'article_id', 'article_published_date',
       'article_text', 'article_title', 'article_url', 'comment_author',
       'comment_counter', 'comment_text','constructive_gold']
attribs['avg_cols'] = ['agree', 'constructive']
attribs['nominal_cols'] = ['constructive_characteristics', 'non_constructive_characteristics']
attribs['text_cols'] = ['crowd_comments', 'other_con_chars', 'other_noncon_chars']

In [8]:
# Get aggregated data
constructiveness_aggregated_df = constructiveness_aggregator.aggregate_annotations(constructiveness_df, attribs)

In [9]:
# Rename some of the columns
constructiveness_aggregated_df.rename(columns = {'agree':'agree_constructiveness_expt', 'crowd_comments':'crowd_comments_constructiveness_expt'}, inplace=True)

In [10]:
# Relevant columns
cols = (['article_id', 'article_author', 'article_published_date', 'article_text', 'article_title',
   'article_url', 'comment_author', 'comment_counter', 'comment_text', 'agree_constructiveness_expt', 'constructive' , 
   'constructive_internal_gold', 
   'constructive_characteristics', 'non_constructive_characteristics', 'other_con_chars', 
   'other_noncon_chars', 'crowd_comments_constructiveness_expt'])

## Write aggregated contructiveness CSV 

In [11]:
constructiveness_aggregator.write_csv(constructiveness_aggregated_df, cols, input_csv.rstrip('.csv') +'_aggregated.csv')

CSV written:  ../../CF_output/constructiveness/batch5_f1282373_aggregated.csv


# Aggregate toxicity annotations

In [12]:
# Read toxicity annotations csv
toxicity_path = '../../CF_output/toxicity/'
crowd_annotations_full = 'batch5_f1282494.csv'
#input_csv = '../../samples/f1270155_toxicity.csv'
#input_csv = toxicity_path + 'batch3_f1271451.csv'
#input_csv = toxicity_path + 'batch4_f1281988.csv'
input_csv = toxicity_path + crowd_annotations_full

In [13]:
# Create a CrowdAggregator object
toxicity_aggregator = CrowdsourceAggregator(input_csv)

In [14]:
# Dataframe with only non gold questions. 
toxicity_df = toxicity_aggregator.get_non_gold_questions()

In [15]:
# Set attributes
attribs = {}
# set attributes 
attribs['unit_id_col'] = ['_unit_id']
attribs['meta_cols'] = ['article_author', 'article_id', 'article_published_date',
       'article_text', 'article_title', 'article_url', 'comment_author',
       'comment_counter', 'comment_text', 'crowd_toxicity_level_gold']
attribs['avg_cols'] = ['agree', 'crowd_toxicity_level']
attribs['nominal_cols'] = ['toxicity_characteristics']
attribs['text_cols'] = ['crowd_comments', 'other_toxic_chars', 'crowd_discard', 'expert_has_content']    

In [16]:
# Get aggregated data
toxicity_aggregated_df = toxicity_aggregator.aggregate_annotations(toxicity_df, attribs)

In [17]:
# Rename some of the columns
toxicity_aggregated_df.rename(columns = {'expert_has_content':'has_content', 'agree':'agree_toxicity_expt', 'crowd_comments':'crowd_comments_toxicity_expt'}, inplace=True)

In [18]:
# relevant columns
cols = (['article_id', 'article_author', 'article_published_date', 'article_text', 'article_title',
   'article_url', 'comment_author', 'comment_counter', 'comment_text', 'agree_toxicity_expt', 'crowd_toxicity_level', 
   'crowd_toxicity_level_gold','has_content', 'crowd_discard',
   'toxicity_characteristics', 'other_toxic_chars', 'crowd_comments_toxicity_expt'])

## Write aggregated contructiveness CSV 

In [19]:
toxicity_aggregator.write_csv(toxicity_aggregated_df, cols, input_csv.rstrip('.csv') +'_aggregated.csv')

CSV written:  ../../CF_output/toxicity/batch5_f1282494_aggregated.csv


## Combine constructiveness and toxicity annotations

In [20]:
dfs = [constructiveness_aggregated_df, toxicity_aggregated_df]

In [21]:
df_merged = reduce(lambda left,right: pd.merge(left,right,on=['article_id', 'article_author', 'article_published_date',
        'article_title', 'article_url', 'article_text',
       'comment_author', 'comment_counter', 'comment_text'], how='outer'), dfs)

In [23]:
# Sort the merged dataframe on constructiveness and toxicity
df_sorted = df_merged.sort_values(by=['constructive', 'crowd_toxicity_level'], ascending = False)

In [24]:
# Relevant columns
cols = (['article_id', 'article_author', 'article_published_date',
        'article_title', 'article_url', 'article_text',
       'comment_author', 'comment_counter', 'comment_text',
       'agree_constructiveness_expt', 'agree_toxicity_expt', 'constructive', 'constructive_gold', 
       'crowd_toxicity_level', 'crowd_toxicity_level_gold',
       'has_content', 'crowd_discard',  
       'constructive_characteristics', 'non_constructive_characteristics',
       'toxicity_characteristics',                 
       'crowd_comments_constructiveness_expt', 
       'crowd_comments_toxicity_expt',
       'other_con_chars', 'other_noncon_chars', 'other_toxic_chars'         
        ])

## Write contructiveness and toxicity combined CSV 

In [25]:
output_dir = '../../CF_output/combined/'

In [26]:
df_sorted.to_csv( output_dir + batch + '_constructiveness_and_toxicity_combined.csv', columns = cols, index = False)